In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
file_path = 'candidates.csv'
data = pd.read_csv(file_path)

# Print the DataFrame
print(data)


        Name  Age  Phone Number College Name  Marks Scored in Unstop   \
0     Dinesh   21    7098438838        Gitam                       86   
1       siva   21    6372872637       anurag                       70   
2  divyamshu   20    6378347434   mallareddy                       90   
3    akilesh   23    6383726288   mallareddy                       89   
4      tanuj   22    6473837373        gitam                       70   
5     ritesh   21   27910192883        mlrit                       77   
6    manohar   20    7893938939       vasavi                       89   
7    rithvik   19    3839922020         mgit                       90   
8  shanawaaz   18    8383939303         cbit                       91   
9      kiran   19    8838339393         jntu                       87   

   Unnamed: 5 Time taken for coding                 Email      Course  \
0         NaN                20mins      dinesh@gmail.com       Btech   
1         NaN                30mins        siva@gm

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

def evaluate_candidates_with_ml(file_path):
    # Read data from CSV
  data = pd.read_csv(file_path)


            # Feature selection
  X = data[['Age', 'Marks Scored in Unstop ']]
  y = (data['Marks Scored in Unstop '] >= 85).astype(int)  # Binary label: 1 if >= 85, else 0

                        # Split data into training and testing sets
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

                                # Train decision tree classifier
  clf = DecisionTreeClassifier(random_state=42)
  clf.fit(X_train, y_train)

                                            # Predict on test set
  y_pred = clf.predict(X_test)

                                                    # Calculate accuracy
  accuracy = accuracy_score(y_test, y_pred)

                                                            # Select candidates with predicted label 1 (>=85)
  selected_candidates = data[data['Marks Scored in Unstop '] >= 85]

  return selected_candidates, accuracy

def main():
 file_path = 'candidates.csv'

 selected_candidates, accuracy = evaluate_candidates_with_ml(file_path)

 if not selected_candidates.empty:
    print("Selected Candidates:")
    print(selected_candidates[['Name','Marks Scored in Unstop ']])


    print(f"Accuracy of the model: {accuracy}")

if __name__ == "__main__":
     main()
























Selected Candidates:
        Name  Marks Scored in Unstop 
0     Dinesh                       86
2  divyamshu                       90
3    akilesh                       89
6    manohar                       89
7    rithvik                       90
8  shanawaaz                       91
9      kiran                       87
Accuracy of the model: 1.0


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import re

def evaluate_candidates_with_ml(file_path):
    # Read data from CSV
 data = pd.read_csv(file_path)
 data['Time taken for coding'] = data['Time taken for coding'].apply(lambda x: re.findall(r'\d+', str(x)))
 data['Time taken for coding'] = data['Time taken for coding'].apply(lambda x: int(x[0]) if x else 0)

            # Feature selection
 X = data[['Age', 'Marks Scored in Unstop ', 'Time taken for coding']]
 y = (data['Marks Scored in Unstop '] >= 85).astype(int)  # Binary label: 1 if >= 85, else 0

                        # Split data into training and testing sets
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

                                # Train decision tree classifier
 clf = make_pipeline(StandardScaler(), DecisionTreeClassifier(random_state=42))
 clf.fit(X_train, y_train)

                                            # Predict on test set
 y_pred = clf.predict(X_test)

                                                    # Calculate accuracy
 accuracy = accuracy_score(y_test, y_pred)

 return accuracy, data

def main():
 file_path = 'candidates.csv'
 accuracy,data = evaluate_candidates_with_ml(file_path)
 print(f"Accuracy of the model: {accuracy}")

                                                                            # Find the candidate who can code in less time
 min_time_index = data['Time taken for coding'].idxmin()
 candidate_name = data.loc[min_time_index, 'Name']
 min_time = data.loc[min_time_index, 'Time taken for coding']
 print(f"The candidate who can code in less time is {candidate_name} with time taken: {min_time}")

if __name__ == "__main__":
    main()


Accuracy of the model: 1.0
The candidate who can code in less time is tanuj with time taken: 11


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

def preprocess_time_column(df):
      df['Time taken for coding'] = df['Time taken for coding'].apply(lambda x: int(x[:-4]))  # Extract numeric part and convert to int
      return df[['Time taken for coding']]

def evaluate_candidates_with_ml(file_path):
    # Read data from CSV
 data = pd.read_csv(file_path)
 time_pipeline = Pipeline([
            ('preprocess_time', FunctionTransformer(preprocess_time_column)),
                    ('imputer', SimpleImputer(strategy='mean'))  # Impute missing values if any
                        ])

                            # Feature selection
 preprocessor = ColumnTransformer([
                                        ('time_pipeline', time_pipeline, ['Time taken for coding']),  # Apply preprocessing to 'Time taken for coding'
                                                ('passthrough', 'passthrough', ['Age', 'Marks Scored in Unstop '])  # Passthrough other features
                                                    ])


 preprocessor = ColumnTransformer([
            ('time_pipeline', time_pipeline, ['Time taken for coding']),  # Apply preprocessing to 'Time taken for coding'
                    ('passthrough', 'passthrough', ['Age', 'Marks Scored in Unstop '])  # Passthrough other features
                        ])


 X = data[['Age', 'Marks Scored in Unstop ', 'Time taken for coding']]
 y = (data['Marks Scored in Unstop '] >= 85).astype(int)  # Binary label: 1 if >= 85, else 0

        # Split data into training and testing sets
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

                # Train decision tree classifier
 clf = Pipeline([
                            ('preprocessor', preprocessor),
                                    ('classifier', DecisionTreeClassifier(random_state=42))
                                        ])
 clf.fit(X_train, y_train)
 y_pred = clf.predict(X_test)

                                                    # Calculate accuracy
 accuracy = accuracy_score(y_test, y_pred)

                                                            # Select candidates with predicted label 1 (>=85)
 selected_candidates = data[data['Marks Scored in Unstop '] >= 85]

 return data, selected_candidates, accuracy

def main():
    file_path = 'candidates.csv'
    data, selected_candidates, accuracy = evaluate_candidates_with_ml(file_path)

    print("Candidates:")
    for index, row in data.iterrows():
      candidate_name = row['Name']
      marks_scored = row['Marks Scored in Unstop ']
      time_taken = row['Time taken for coding']

      selected_status = "Selected" if candidate_name in selected_candidates['Name'].values else "Not Selected"
      print(f"Name: {candidate_name}, Marks Scored: {marks_scored}, Time Taken: {time_taken}, Selection Status: {selected_status}")

    print(f"\nAccuracy of the model: {accuracy}")

if __name__ == "__main__":
     main()


Candidates:
Name: Dinesh, Marks Scored: 86, Time Taken: 20mins, Selection Status: Selected
Name: siva, Marks Scored: 70, Time Taken: 30mins, Selection Status: Not Selected
Name: divyamshu, Marks Scored: 90, Time Taken: 40mins, Selection Status: Selected
Name: akilesh, Marks Scored: 89, Time Taken: 12mins, Selection Status: Selected
Name: tanuj, Marks Scored: 70, Time Taken: 11mins, Selection Status: Not Selected
Name: ritesh, Marks Scored: 77, Time Taken: 25mins, Selection Status: Not Selected
Name: manohar, Marks Scored: 89, Time Taken: 30mins, Selection Status: Selected
Name: rithvik, Marks Scored: 90, Time Taken: 60mins, Selection Status: Selected
Name: shanawaaz, Marks Scored: 91, Time Taken: 42mins, Selection Status: Selected
Name: kiran, Marks Scored: 87, Time Taken: 53mins, Selection Status: Selected

Accuracy of the model: 1.0


In [ ]:
import pandas as pd
import datetime

# Load candidate data from CSV file
candidate_data = pd.read_csv('candidates.csv')
selected_candidates = candidate_data[candidate_data[ 'Marks Scored in Unstop '] >= 85]

# Sample interview scheduling details
interview_details = {
'date': datetime.date(2024, 3, 21),
'time_slots': ['10:00 AM', '2:00 PM'],
'interviewer': 'John Doe',
            }

            # Check technical compatibility
def check_technical_compatibility(candidate_row):
                # Assume candidate has the required technical equipment
   return True

                    # Set up virtual interview environment
def setup_virtual_interview(interview_details, candidate_row):
   print("Setting up virtual interview for", candidate_row['Name'], "with", interview_details['interviewer'])
   print("Date:", interview_details['date'])
   print("Time slots:", interview_details['time_slots'])
   print("Technical requirements: Laptop with webcam, Stable internet connection")
   print("Candidate details:", candidate_row[['Name', 'Age', 'Phone Number', 'Marks Scored in Unstop ', 'Time taken for coding']])
   print("Environment configured successfully.")

                                            # Send invitations
def send_invitations(interview_details, candidate_row):
   print("Sending interview invitation to", candidate_row['Name'], "and", interview_details['interviewer'])
   print("Invitation sent successfully.")

                                                    # Execute Virtual Interview Setup Algorithm
def execute_virtual_interview_setup(interview_details, candidate_data):
   for index, candidate_row in candidate_data.iterrows():
                                                                # Check technical compatibility
       is_compatible = check_technical_compatibility(candidate_row)

       if not is_compatible:
           print("Technical compatibility issues found for", candidate_row['Name'], ". Unable to proceed with the interview setup.")
           continue

                                                                                                                # Set up virtual interview environment
       setup_virtual_interview(interview_details, candidate_row)

                                                                                                                                # Send invitations
       send_invitations(interview_details, candidate_row)
       print()  # Print empty line for better readability

                                                                                                                                                # Execute the algorithm
execute_virtual_interview_setup(interview_details, candidate_data)


Setting up virtual interview for Dinesh with John Doe
Date: 2024-03-21
Time slots: ['10:00 AM', '2:00 PM']
Technical requirements: Laptop with webcam, Stable internet connection
Candidate details: Name                           Dinesh
Age                                21
Phone Number               7098438838
Marks Scored in Unstop             86
Time taken for coding          20mins
Name: 0, dtype: object
Environment configured successfully.
Sending interview invitation to Dinesh and John Doe
Invitation sent successfully.

Setting up virtual interview for siva with John Doe
Date: 2024-03-21
Time slots: ['10:00 AM', '2:00 PM']
Technical requirements: Laptop with webcam, Stable internet connection
Candidate details: Name                             siva
Age                                21
Phone Number               6372872637
Marks Scored in Unstop             70
Time taken for coding          30mins
Name: 1, dtype: object
Environment configured successfully.
Sending interview invitat

In [ ]:
import pandas as pd

# Load candidate data from CSV
candidate_data = pd.read_csv('candidates.csv')

# Display resumes of candidates
for index, candidate in candidate_data.iterrows():
 print("Candidate Name:", candidate['Name'])
 print("Email:", candidate['Email'])
 print("Phone Number:", candidate['Phone Number'])
 print("Education:")
 print(candidate['Course'])
 print("Course"":")
 print(candidate['Experience'])
 print("Skills:")
 print(candidate['Skills'])
 print("")




Candidate Name: Dinesh
Email: dinesh@gmail.com
Phone Number: 7098438838
Education:
Btech
Course:
0 yrs
Skills:
java,python

Candidate Name: siva
Email: siva@gmail.com
Phone Number: 6372872637
Education:
BA
Course:
1yr
Skills:
django,python

Candidate Name: divyamshu
Email: divmyanshu@gmail.com
Phone Number: 6378347434
Education:
BBA
Course:
1yr
Skills:
c,java,python

Candidate Name: akilesh
Email: akilesh@gmail.com
Phone Number: 6383726288
Education:
B arch
Course:
2yrs
Skills:
python

Candidate Name: tanuj
Email: tanuj@gmail.com
Phone Number: 6473837373
Education:
Btech
Course:
2yrs
Skills:
c++,reactjs

Candidate Name: ritesh
Email: ritesh@gmail.com
Phone Number: 27910192883
Education:
Btech
Course:
1yr
Skills:
html,css,javascript

Candidate Name: manohar
Email: manohar@gmail.com
Phone Number: 7893938939
Education:
Pyschology
Course:
1yr
Skills:
ai and ml

Candidate Name: rithvik
Email: ritvik@gmail.com
Phone Number: 3839922020
Education:
BBA
Course:
1yr
Skills:
html,js,angularjs

Can